In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import lognorm

In [ ]:
# This is the ACS County-level data. I got them from this website: https://data.census.gov/table/ACSDP1Y2022.DP03?q=DP03&g=010XX00US$0500000.
# Again, please replace with the actual path to the file
df = pd.read_csv('synthetic_dataset.csv')

filtered_df = df[df['mastercard_si_abbr'].astype(str).str.startswith(('44', '45'))]

# This is the ACS County-level data. I got them from this website: https://data.census.gov/table/ACSDP1Y2022.DP03?q=DP03&g=010XX00US$0500000.
# Again, please replace with the actual path to the file
income_data = pd.read_csv('ACSDP1Y2022.DP03-Data.csv')

filtered_df['date'] = pd.to_datetime(filtered_df['date'])
filtered_df = filtered_df[filtered_df['date'] >= pd.Timestamp('2019-01-01')]
income_data['extracted_fips'] = income_data['GEO_ID'].str[-5:]
merged_df = pd.merge(filtered_df, income_data[['extracted_fips', 'DP03_0062E']],
                     left_on='county_fips', right_on='extracted_fips', how='inner')
merged_df.rename(columns={'DP03_0062E': 'median_income'}, inplace=True)
merged_df.drop('extracted_fips', axis=1, inplace=True)
merged_df['median_income'] = pd.to_numeric(merged_df['median_income'], errors='coerce').dropna()

def analyze_spending_and_card_count_for_quartile(quartile):
    quartile_str = str(quartile).strip()
    df_filtered = merged_df[merged_df['income_quartile'].astype(str).str.strip() == quartile_str]
    if df_filtered.empty:
        print(f"No data available for quartile {quartile}. Skipping analysis.")
        return
    df_filtered = df_filtered.sort_values(by='date')
    aggregated = df_filtered.resample('M', on='date').agg({'total_spend': 'sum', 'card_count': 'sum'})
    if aggregated.empty:
        print(f"No aggregated data available for quartile {quartile}. Skipping analysis.")
        return
    initial_card_count = aggregated.iloc[0]['card_count']

    aggregated['adjusted_spending'] = aggregated['total_spend'] * (initial_card_count / aggregated['card_count'])

    monthly_spending_change = aggregated['total_spend'].pct_change() * 100
    adjusted_monthly_spending_change = aggregated['adjusted_spending'].pct_change() * 100

    yearly_spending_change = aggregated['total_spend'].pct_change(periods=12) * 100
    adjusted_yearly_spending_change = aggregated['adjusted_spending'].pct_change(periods=12) * 100

    adjusted_monthly_spending_change.plot(kind='bar', figsize=(12, 6), title=f'Adjusted Monthly Spending Change for Quartile {quartile}')
    plt.xlabel('Month')
    plt.ylabel('Adjusted Spending Change (%)')
    plt.xticks(rotation=45)
    plt.show()
    
    adjusted_yearly_spending_change.plot(kind='bar', figsize=(12, 6), title=f'Adjusted Yearly Spending Change for Quartile {quartile}')
    plt.xlabel('Month')
    plt.ylabel('Adjusted Spending Change (%)')
    plt.xticks(rotation=45)
    plt.show()
    
    print(f"Adjusted Monthly Spending Change for Quartile {quartile} (cleaned):\n", adjusted_monthly_spending_change.dropna())
    print(f"Adjusted Yearly Spending Change for Quartile {quartile} (cleaned):\n", adjusted_yearly_spending_change.dropna())

for quartile in range(1, 5):
    analyze_spending_and_card_count_for_quartile(quartile)

In [122]:
import pandas as pd

data = pd.read_csv('cc-est2022-all.csv', encoding = 'latin1')
county_codes_df = pd.read_csv('countyfipstool20190120.csv')

def map_age_group(age_group):
    if 1 <= age_group <= 4:
        return '1-4'
    elif 5 <= age_group <= 8:
        return '5-8'
    elif 9 <= age_group <= 13:
        return '9-13'
    elif 14 <= age_group <= 18:
        return '14-18'
    else:
        return 'Other'

data['AGE_CATEGORY'] = data['AGEGRP'].apply(map_age_group)

data = data[data['AGE_CATEGORY'] != 'Other']

grouped_data = data.groupby(['STNAME', 'AGE_CATEGORY']).agg({
    'TOT_POP': 'sum',
    'WA_MALE': 'sum', 'WA_FEMALE': 'sum',
    'BA_MALE': 'sum', 'BA_FEMALE': 'sum',
    'IA_MALE': 'sum', 'IA_FEMALE': 'sum',
    'AA_MALE': 'sum', 'AA_FEMALE': 'sum',
    'NA_MALE': 'sum', 'NA_FEMALE': 'sum',
    'H_MALE': 'sum', 'H_FEMALE': 'sum'
}).reset_index()

data = data.groupby(['CTYNAME', 'AGE_CATEGORY']).agg({
    'TOT_POP': 'sum',
    'WA_MALE': 'sum', 'WA_FEMALE': 'sum',
    'BA_MALE': 'sum', 'BA_FEMALE': 'sum',
    'IA_MALE': 'sum', 'IA_FEMALE': 'sum',
    'AA_MALE': 'sum', 'AA_FEMALE': 'sum',
    'NA_MALE': 'sum', 'NA_FEMALE': 'sum',
    'H_MALE': 'sum', 'H_FEMALE': 'sum'
}).reset_index()

data['WHITE_NUM'] = (data['WA_MALE'] + data['WA_FEMALE'])
data['BLACK_NUM'] = (data['BA_MALE'] + data['BA_FEMALE'])
data['HISPANIC_NUM'] = (data['H_MALE'] + data['H_FEMALE'])
data['ASIAN_NUM'] = (data['AA_MALE'] + data['AA_FEMALE'])
data['NATIVE_NUM'] = (data['IA_MALE'] + data['IA_FEMALE'])
data['NATIVE_HAWAIIAN_NUM'] = (data['NA_MALE'] + data['NA_FEMALE'])

columns_to_drop_data = ['WA_MALE', 'WA_FEMALE', 'BA_MALE', 'BA_FEMALE',
                   'H_MALE', 'H_FEMALE', 'AA_MALE', 'AA_FEMALE',
                   'IA_MALE', 'IA_FEMALE', 'NA_MALE', 'NA_FEMALE', 'sab', 'sid', 'saint', 'cfips', 'cname']

combined_df = pd.merge(data, county_codes_df, left_on='CTYNAME', right_on='cname')
combined_df.drop(columns_to_drop_data, axis=1, inplace=True)
aggregated_data = merged_df.groupby('county_fips')['card_count'].sum().reset_index()
aggregated_data['county_fips'] = aggregated_data['county_fips'].astype(int)
combined_df['fips'] = combined_df['fips'].astype(int)
combined_df['county_total_population'] = combined_df.groupby('fips')['TOT_POP'].transform('sum')

combined_df = pd.merge(combined_df, aggregated_data, how='left', left_on='fips', right_on='county_fips')

combined_df['card_count'] = combined_df['card_count'].fillna(0)

combined_card_df = combined_df.copy()

demographic_columns = ['WHITE_NUM', 'BLACK_NUM', 'HISPANIC_NUM', 'ASIAN_NUM', 'NATIVE_NUM', 'NATIVE_HAWAIIAN_NUM']

for column in demographic_columns:
    combined_card_df[column] = (combined_card_df[column] / combined_card_df['county_total_population']) * combined_card_df['card_count']
    
columns_to_drop = ['county_total_population', 'card_count', 'sfips', 'county_fips', 'fips']
# combined_df.drop(columns_to_drop, axis=1, inplace=True)
# combined_card_df.drop(columns_to_drop, axis=1, inplace=True)

In [107]:
combined_df

CTYNAME AGE_CATEGORY  TOT_POP  WHITE_NUM  BLACK_NUM  \
0      Abbeville County          1-4    21980      15110       5842   
1      Abbeville County        14-18    22031      16379       5317   
2      Abbeville County          5-8    21880      15441       5938   
3      Abbeville County         9-13    31266      22106       8657   
4         Acadia Parish          1-4    64178      47616      13422   
...                 ...          ...      ...        ...        ...   
12099     Zavala County         9-13    10167       9776        140   
12100    Ziebach County          1-4     2943        619         26   
12101    Ziebach County        14-18     1005        419          4   
12102    Ziebach County          5-8     2652        565         12   
12103    Ziebach County         9-13     3000        906         16   

       HISPANIC_NUM  ASIAN_NUM  NATIVE_NUM  NATIVE_HAWAIIAN_NUM  \
0               641         91         108                   18   
1               157         75          67                    0   
2               557         68          79                    0   
3               476        113         149                    9   
4              2844        247         286                   38   
...             ...        ...         ...                  ...   
12099          9531         25         155                   16   
12100           244          8        2099                    0   
12101             7          0         558                    0   
12102           123          8        1920                    1   
12103            44          8        2021                    4   

                sname  sfips   fips  county_total_population  county_fips  \
0      South Carolina     45  45001                    97157          NaN   
1      South Carolina     45  45001                    97157          NaN   
2      South Carolina     45  45001                    97157          NaN   
3      South Carolina     45  45001                    97157          NaN   
4           Louisiana     22  22001                   229047          NaN   
...               ...    ...    ...                      ...          ...   
12099           Texas     48  48507                    38196          NaN   
12100    South Dakota     46  46137                     9600          NaN   
12101    South Dakota     46  46137                     9600          NaN   
12102    South Dakota     46  46137                     9600          NaN   
12103    South Dakota     46  46137                     9600          NaN   

       card_count  
0             0.0  
1             0.0  
2             0.0  
3             0.0  
4             0.0  
...           ...  
12099         0.0  
12100         0.0  
12101         0.0  
12102         0.0  
12103         0.0  

[12104 rows x 15 columns]

In [123]:
combined_card_df.dropna(subset = ['county_fips'])

CTYNAME AGE_CATEGORY  TOT_POP  WHITE_NUM  BLACK_NUM  HISPANIC_NUM  \
12       Ada County          1-4   507369  50.407677   1.077918      7.276954   
13       Ada County        14-18   317803  34.362328   0.123978      1.088343   
14       Ada County          5-8   557200  56.099141   1.141364      7.216982   
15       Ada County         9-13   642673  67.252048   0.759340      4.962278   
32     Adams County          1-4   933402  55.990074   3.872488     26.266181   
...             ...          ...      ...        ...        ...           ...   
12083   Yuba County         9-13    91429  41.618623   2.312240     12.939846   
12084   Yuma County          1-4   241701  61.113668   1.698995     53.324047   
12086   Yuma County        14-18   171355  45.980803   0.549517     15.164427   
12088   Yuma County          5-8   235753  59.730011   2.295010     48.006408   
12090   Yuma County         9-13   213493  55.084757   1.552175     38.309257   

       ASIAN_NUM  NATIVE_NUM  NATIVE_HAWAIIAN_NUM       sname  sfips   fips  \
12      1.557914    0.543891             0.133058       Idaho     16  16001   
13      0.651055    0.158280             0.029593       Idaho     16  16001   
14      2.152361    0.634801             0.205585       Idaho     16  16001   
15      2.110773    0.525059             0.143820       Idaho     16  16001   
32      2.028074    1.646058             0.158426    Colorado      8   8001   
...          ...         ...                  ...         ...    ...    ...   
12083   3.587530    1.563896             0.355816  California      6   6115   
12084   0.905154    1.975727             0.185709     Arizona      4   4027   
12086   0.556844    0.626450             0.079187     Arizona      4   4027   
12088   1.004631    1.719003             0.182609     Arizona      4   4027   
12090   1.173431    1.325605             0.192190     Arizona      4   4027   

       county_total_population  county_fips  card_count  
12                     2025045      16001.0       227.0  
13                     2025045      16001.0       227.0  
14                     2025045      16001.0       227.0  
15                     2025045      16001.0       227.0  
32                     3473472       8001.0       249.0  
...                        ...          ...         ...  
12083                   331098       6115.0       187.0  
12084                   862302       4027.0       243.0  
12086                   862302       4027.0       243.0  
12088                   862302       4027.0       243.0  
12090                   862302       4027.0       243.0  

[3176 rows x 15 columns]

In [78]:
aggregated_data

county_fips  card_count
0         01003         225
1         01015         167
2         01043         206
3         01049         228
4         01051         198
..          ...         ...
821       55133         229
822       55139         239
823       55141         241
824       56021         195
825       56025         185

[826 rows x 2 columns]

In [96]:
data = pd.read_csv('cc-est2022-all.csv', encoding = 'latin1')

def map_age_group(age_group):
    if 1 <= age_group <= 4:
        return '1-4'
    elif 5 <= age_group <= 8:
        return '5-8'
    elif 9 <= age_group <= 13:
        return '9-13'
    elif 14 <= age_group <= 18:
        return '14-18'
    else:
        return 'Other'

data['AGE_CATEGORY'] = data['AGEGRP'].apply(map_age_group)

data = data[data['AGE_CATEGORY'] != 'Other']

grouped_data = data.groupby(['STNAME', 'AGE_CATEGORY']).agg({
    'TOT_POP': 'sum',
    'WA_MALE': 'sum', 'WA_FEMALE': 'sum',
    'BA_MALE': 'sum', 'BA_FEMALE': 'sum',
    'IA_MALE': 'sum', 'IA_FEMALE': 'sum',
    'AA_MALE': 'sum', 'AA_FEMALE': 'sum',
    'NA_MALE': 'sum', 'NA_FEMALE': 'sum',
    'H_MALE': 'sum', 'H_FEMALE': 'sum'
}).reset_index()

data = data.groupby(['CTYNAME', 'AGE_CATEGORY']).agg({
    'TOT_POP': 'sum',
    'WA_MALE': 'sum', 'WA_FEMALE': 'sum',
    'BA_MALE': 'sum', 'BA_FEMALE': 'sum',
    'IA_MALE': 'sum', 'IA_FEMALE': 'sum',
    'AA_MALE': 'sum', 'AA_FEMALE': 'sum',
    'NA_MALE': 'sum', 'NA_FEMALE': 'sum',
    'H_MALE': 'sum', 'H_FEMALE': 'sum'
}).reset_index()

grouped_data['WHITE_NUM'] = (grouped_data['WA_MALE'] + grouped_data['WA_FEMALE'])
grouped_data['BLACK_NUM'] = (grouped_data['BA_MALE'] + grouped_data['BA_FEMALE'])
grouped_data['HISPANIC_NUM'] = (grouped_data['H_MALE'] + grouped_data['H_FEMALE'])
grouped_data['ASIAN_NUM'] = (grouped_data['AA_MALE'] + grouped_data['AA_FEMALE'])
grouped_data['NATIVE_NUM'] = (grouped_data['IA_MALE'] + grouped_data['IA_FEMALE'])
grouped_data['NATIVE_HAWAIIAN_NUM'] = (grouped_data['NA_MALE'] + grouped_data['NA_FEMALE'])

data['WHITE_NUM'] = (data['WA_MALE'] + data['WA_FEMALE'])
data['BLACK_NUM'] = (data['BA_MALE'] + data['BA_FEMALE'])
data['HISPANIC_NUM'] = (data['H_MALE'] + data['H_FEMALE'])
data['ASIAN_NUM'] = (data['AA_MALE'] + data['AA_FEMALE'])
data['NATIVE_NUM'] = (data['IA_MALE'] + data['IA_FEMALE'])
data['NATIVE_HAWAIIAN_NUM'] = (data['NA_MALE'] + data['NA_FEMALE'])

In [108]:
county_codes_df

sname sab   sid  sfips           cname  saint  cfips   fips
0      Alabama  AL   1.0      1  AUTAUGA COUNTY      0      1   1001
1      Alabama  AL   1.0      1    AUTAUGA CTY.      0      1   1001
2      Alabama  AL   1.0      1    autauga cty.      0      1   1001
3      Alabama  AL   1.0      1   AUTAUGA CNTY.      0      1   1001
4      Alabama  AL   1.0      1      AUTAUGA CY      0      1   1001
...        ...  ..   ...    ...             ...    ...    ...    ...
77608  Wyoming  WY  50.0     56       weston cy      0     45  56045
77609  Wyoming  WY  50.0     56          Weston      0     45  56045
77610  Wyoming  WY  50.0     56   Weston County      0     45  56045
77611  Wyoming  WY  50.0     56      WESTON CTY      0     45  56045
77612  Wyoming  WY  50.0     56      weston cy.      0     45  56045

[77613 rows x 8 columns]

In [98]:
data = pd.read_csv('cc-est2022-all.csv', encoding = 'latin1')
data

SUMLEV  STATE  COUNTY   STNAME         CTYNAME  YEAR  AGEGRP  TOT_POP  \
0           50      1       1  Alabama  Autauga County     1       0    58802   
1           50      1       1  Alabama  Autauga County     1       1     3443   
2           50      1       1  Alabama  Autauga County     1       2     3613   
3           50      1       1  Alabama  Autauga County     1       3     4131   
4           50      1       1  Alabama  Autauga County     1       4     3963   
...        ...    ...     ...      ...             ...   ...     ...      ...   
238939      50     56      45  Wyoming   Weston County     4      14      590   
238940      50     56      45  Wyoming   Weston County     4      15      415   
238941      50     56      45  Wyoming   Weston County     4      16      254   
238942      50     56      45  Wyoming   Weston County     4      17      141   
238943      50     56      45  Wyoming   Weston County     4      18      170   

        TOT_MALE  TOT_FEMALE  ...  HWAC_MALE  HWAC_FEMALE  HBAC_MALE  \
0          28761       30041  ...        854          734        112   
1           1790        1653  ...         83           54         14   
2           1849        1764  ...         81           68          5   
3           2121        2010  ...         81           81         10   
4           2003        1960  ...         72           70          8   
...          ...         ...  ...        ...          ...        ...   
238939       323         267  ...          5            7          0   
238940       231         184  ...          6            5          0   
238941       127         127  ...          2            0          0   
238942        71          70  ...          1            1          0   
238943        58         112  ...          0            2          0   

        HBAC_FEMALE  HIAC_MALE  HIAC_FEMALE  HAAC_MALE  HAAC_FEMALE  \
0                97         40           30         22           24   
1                 9          3            1          9            1   
2                 8          7            3          0            1   
3                10          1            3          2            2   
4                 9          5            2          2            4   
...             ...        ...          ...        ...          ...   
238939            1          0            0          0            0   
238940            0          0            0          0            0   
238941            0          0            0          0            0   
238942            0          1            0          0            0   
238943            0          0            0          0            0   

        HNAC_MALE  HNAC_FEMALE  
0              19           11  
1               3            1  
2               1            2  
3               2            1  
4               3            2  
...           ...          ...  
238939          0            0  
238940          0            0  
238941          0            0  
238942          0            0  
238943          0            0  

[238944 rows x 80 columns]